# 📘 Enhanced Error Messages in NLSQ> Intelligent, actionable error messages for faster debugging⏱️ **10-15 minutes** | 📊 **Level: ●○○ Beginner** | 🏷️ **Feature Demo**---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/05_feature_demos/enhanced_error_messages_demo.ipynb)


In [ ]:
# @title Install NLSQ (run once in Colab)
import sys
if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("✅ NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

## 🎯 Learning ObjectivesAfter this tutorial, you'll be able to:1. Understand NLSQ's enhanced error diagnostics2. Use error recommendations to fix optimization failures3. Implement programmatic error handling4. Appreciate the improvement over generic error messages---

## 🔬 Feature Overview**What problem does this solve?**Traditional optimization libraries give cryptic errors:```RuntimeError: Optimal parameters not found:CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH```NLSQ provides:- Clear failure reasons in plain English- Specific, actionable recommendations- Detailed diagnostic information- Programmatic error handling capabilities---

## Setup

In [1]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [2]:
import jax.numpy as jnp
import numpy as np

from nlsq import curve_fit
from nlsq.utils.error_messages import OptimizationError

## Example 1: Maximum Iterations ReachedSee how NLSQ explains common failures with actionable advice.

In [3]:
def exponential(x, a, b):
    return a * jnp.exp(-b * x)


# Generate data
x = np.linspace(0, 5, 50)
y = 3 * np.exp(-0.5 * x) + np.random.normal(0, 0.1, 50)

try:
    # Force failure with very low max_nfev
    popt, pcov = curve_fit(exponential, x, y, p0=[1, 1], max_nfev=3)
except OptimizationError as e:
    print('❌ Optimization Failed!')
    print(f'\n{e}')
    print('\n📊 Diagnostic Details:')
    for key, value in e.diagnostics.items():
        print(f'  • {key}: {value}')
    print('\n💡 Recommendations:')
    for i, rec in enumerate(e.recommendations, 1):
        print(f'  {i}. {rec}')

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 2, 'n_data_points': 50, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 2, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 2, 'n_residuals': 50, 'max_nfev': 3}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=2.870903e+01 | ‖∇f‖=1.604196e+01 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=8.410311e+00 | ‖∇f‖=8.508844e+01 | step=2.828427e+00 | nfev=2


...

INFO:nlsq.least_squares:Convergence: reason=The maximum number of function evaluations is exceeded. | iterations=2 | final_cost=6.518969e-01 | time=1.388s | final_gradient_norm=9.464645543744057


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 1.772122s


[ERROR] Optimization failed | {'reason': 'The maximum number of function evaluations is exceeded.', 'status': 0}


ERROR:nlsq.curve_fit:Optimization failed | {'reason': 'The maximum number of function evaluations is exceeded.', 'status': 0}


❌ Optimization Failed!

Optimization failed to converge.

Diagnostics:
  - Final cost: 6.518969e-01
  - Gradient norm: 9.464646e+00
  - Gradient tolerance: 1.000000e-08
  - Function evaluations: 3 / 3
  - Iterations: 2
  - Status: The maximum number of function evaluations is exceeded.

Reasons:
  - Gradient norm 9.46e+00 exceeds tolerance 1.00e-08
  - Reached maximum function evaluations (3)

Recommendations:
  ✓ Try looser gradient tolerance: gtol=1.0e-07
  ✓ Check if initial guess p0 is reasonable
  ✓ Consider parameter scaling with x_scale
  ✓ Increase iteration limit: max_nfev=6
  ✓ Provide better initial guess p0
  ✓ Try different optimization method (trf/dogbox/lm)

For more help, see: https://nlsq.readthedocs.io/troubleshooting

📊 Diagnostic Details:
  • Final cost: 6.518969e-01
  • Gradient norm: 9.464646e+00
  • Gradient tolerance: 1.000000e-08
  • Function evaluations: 3 / 3
  • Iterations: 2
  • Status: The maximum number of function evaluations is exceeded.

💡 Recommendati

## Example 2: Applying RecommendationsShow how following recommendations leads to success.

In [4]:
def exponential(x, a, b):
    return a * jnp.exp(-b * x)


x = np.linspace(0, 5, 50)
y = 3 * np.exp(-0.5 * x) + np.random.normal(0, 0.1, 50)

# First attempt: fails
print('🔴 First attempt (max_nfev=3):')
try:
    popt, pcov = curve_fit(exponential, x, y, p0=[1, 1], max_nfev=3)
    print('  ✅ Succeeded (unexpected)')
except OptimizationError as e:
    print(f'  ❌ Failed: {e.reasons[0] if e.reasons else "Unknown"}')
    print(f'  💡 Recommendation: {e.recommendations[0] if e.recommendations else "Increase max_nfev"}')

# Second attempt: apply recommendation
print('\n🟢 Second attempt (max_nfev=100):')
try:
    popt, pcov = curve_fit(exponential, x, y, p0=[1, 1], max_nfev=100)
    print(f'  ✅ Success! Fitted: a={popt[0]:.3f}, b={popt[1]:.3f}')
    print('  📈 True parameters: a=3.000, b=0.500')
except OptimizationError as e:
    print(f'  ❌ Still failed: {e.reasons[0]}')

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 2, 'n_data_points': 50, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 2, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


🔴 First attempt (max_nfev=3):


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 2, 'n_residuals': 50, 'max_nfev': 3}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=2.846712e+01 | ‖∇f‖=1.603441e+01 | nfev=1


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.281009s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=6 | final_cost=1.613383e-01 | time=0.281s | final_gradient_norm=5.277158303890542e-07


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.488831s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.4888314989984792, 'final_cost': 0.3226765173231921, 'covariance_warning': False}


  ✅ Success! Fitted: a=3.084, b=0.519
  📈 True parameters: a=3.000, b=0.500


## Example 3: Programmatic Error HandlingUse error diagnostics to implement automatic retry strategies.

In [5]:
def gaussian(x, amp, mu, sigma):
    return amp * jnp.exp(-((x - mu)**2) / (2*sigma**2))


x = np.linspace(-5, 5, 100)
y = 2 * np.exp(-((x - 1)**2) / (2*0.5**2))

try:
    popt, pcov = curve_fit(gaussian, x, y, p0=[1, 0, 1], max_nfev=2)
except OptimizationError as e:
    print('📊 Analyzing Error Diagnostics:')
    print(f'  • Number of reasons: {len(e.reasons)}')
    print(f'  • Number of recommendations: {len(e.recommendations)}')

    # Programmatic decision making
    if any('maximum' in r.lower() for r in e.reasons):
        print('\n🔧 Auto-fix strategy: Increase max_nfev')
        try:
            popt, pcov = curve_fit(gaussian, x, y, p0=[1, 0, 1], max_nfev=200)
            print('  ✅ Auto-retry succeeded!')
            print(f'     Fitted: amp={popt[0]:.2f}, mu={popt[1]:.2f}, sigma={popt[2]:.2f}')
        except OptimizationError:
            print('  ❌ Auto-retry failed')

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': 2}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=1.144267e+01 | ‖∇f‖=1.190261e+01 | nfev=1


[WARNING] Maximum number of function evaluations reached | {'nfev': 2}


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.286823s


INFO:nlsq.least_squares:Convergence: reason=`xtol` termination condition is satisfied. | iterations=8 | final_cost=6.548162e-33 | time=0.287s | final_gradient_norm=2.681925049301261e-16


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.539257s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.5392570090007212, 'final_cost': 1.3096323621833204e-32, 'covariance_warning': False}


  ✅ Auto-retry succeeded!
     Fitted: amp=2.00, mu=1.00, sigma=0.50


## Example 4: Before vs After ComparisonSee the dramatic improvement in error message quality.

In [6]:
def difficult(x, a, b, c):
    return a * jnp.sin(b * x) * jnp.exp(-c * x)


x = np.linspace(0, 10, 50)
y = 2 * np.sin(3 * x) * np.exp(-0.5 * x)

print('🔴 OLD ERROR (before enhancement):')
print('  "RuntimeError: Optimal parameters not found:')
print('   CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH"')
print('\n  😕 Not helpful! What should I do?')

print('\n🟢 NEW ERROR (with enhancement):')
try:
    popt, pcov = curve_fit(difficult, x, y, p0=[1, 1, 1], max_nfev=3)
except OptimizationError as e:
    print(f'\n{e}')

print('\n  ✅ Much better! Clear diagnostics and actionable steps!')

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 50, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


🔴 OLD ERROR (before enhancement):
  "RuntimeError: Optimal parameters not found:
   CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH"

  😕 Not helpful! What should I do?

🟢 NEW ERROR (with enhancement):


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 50, 'max_nfev': 3}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=4.299615e+00 | ‖∇f‖=6.293683e-01 | nfev=1


...

INFO:nlsq.least_squares:Convergence: reason=The maximum number of function evaluations is exceeded. | iterations=2 | final_cost=3.185493e+00 | time=0.287s | final_gradient_norm=0.7199105500094608


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.667619s


[ERROR] Optimization failed | {'reason': 'The maximum number of function evaluations is exceeded.', 'status': 0}


ERROR:nlsq.curve_fit:Optimization failed | {'reason': 'The maximum number of function evaluations is exceeded.', 'status': 0}



Optimization failed to converge.

Diagnostics:
  - Final cost: 3.185493e+00
  - Gradient norm: 7.199106e-01
  - Gradient tolerance: 1.000000e-08
  - Function evaluations: 3 / 3
  - Iterations: 2
  - Status: The maximum number of function evaluations is exceeded.

Reasons:
  - Gradient norm 7.20e-01 exceeds tolerance 1.00e-08
  - Reached maximum function evaluations (3)

Recommendations:
  ✓ Try looser gradient tolerance: gtol=1.0e-07
  ✓ Check if initial guess p0 is reasonable
  ✓ Consider parameter scaling with x_scale
  ✓ Increase iteration limit: max_nfev=6
  ✓ Provide better initial guess p0
  ✓ Try different optimization method (trf/dogbox/lm)

For more help, see: https://nlsq.readthedocs.io/troubleshooting

  ✅ Much better! Clear diagnostics and actionable steps!


## 💡 Key Insights1. **Enhanced errors** provide clear explanations instead of cryptic codes2. **Diagnostics** give detailed information about what went wrong3. **Recommendations** are specific and actionable4. **Programmatic access** enables automatic error recovery5. **Much faster debugging** compared to traditional error messages---## 📚 Error Types DetectedNLSQ provides enhanced messages for:- Maximum iterations/evaluations reached- Convergence failures- Numerical instabilities- Invalid initial parameters- Singular covariance matrices- And more...---## 🎓 Best Practices- Always catch `OptimizationError` for robust code- Use `e.diagnostics` for detailed troubleshooting- Implement retry logic using `e.recommendations`- Log error details for debugging production issues- Share error messages when asking for help (they're now useful!)---